In [23]:
import numpy as np
from scipy import stats
from operator import itemgetter

Step1: create team names to feed into pools
Step2: Create pools from list of team names
Step3: Add games
Step4: resolve head-to-head ties (if any)
Step5: Print rankings

In [37]:
class pool: 
    def __init__(self, court_num, team_names):
        self.court = court_num ## name of court e.g. M1, W1, etc
        self.teams = team_names ## list/tuple of team names
        self.points = np.zeros(len(self.teams), dtype=np.int)
        self.HTH = {} #dictionary to track which team won HTH
        self.games_won = np.zeros(len(self.teams), dtype=np.int) #array with number of games_won for each team. indices correspond with self.teams
        self.notes = '' #place for notes - which team was working when they lost the ball, etc
        self.rankings = [] #place for rankings (after the pool is done playing)
        self.rankings_sorted = [] #place for sorted rankings
        
    # method to get which teams are in this pool
    def get_teams(self):
        return self.teams
    
    # method to get team's index within this pool
    def get_teamindex(self,team_name):
        return self.teams.index(team_name)
    
    # method to get the team's total points
    # if team_name = 0 then the entire array of points for all the teams are returned
    # otherwise, it will only show points for a specific team
    def get_points(self, team_name=0):
        if team_name==0:
            return self.points
        else:
            if team_name in self.teams:
                i = self.get_teamindex(team_name)
                return self.points[i]
            else:
                return "Team not found"
    
    # returns the number of games won
    # if team_name = 0, then games won for all the teams are returned
    # otherwise, only games won for a specific team is returned
    def get_gameswon(self, team_name=0):
        if team_name==0:
            return self.games_won
        else:
            if team_name in self.teams:
                i = self.get_teamindex(team_name)
                return self.games_won[i]
            else:
                return "Team not found"
        
    # method to add notes to this pool
    def add_notes(self, comment):
        self.notes += comment + "\n"
    
    # method to return the notes within this pool
    def get_notes(self):
        return self.notes
        
    # inputs: 
    # team_names - list of team names
    # scores - list of scores as lists, in order of team names
    # no output
    # updates games_won, points, and HTH
    def add_games(self, team_names, scores): 
        game_results = {team_names[0]:0, team_names[1]:0}
        pt_results = {team_names[0]:0, team_names[1]:0}
        
        # decides who won and updates games_won
        for score in scores:
            diff = score[1] - score[0]
            if diff > 0:
                game_results[team_names[1]] += 1
            elif diff < 0:
                game_results[team_names[0]] += 1
            else:
                print ("Scores are the same: " + score[0] + ":" + score[1])
            
            pt_results[team_names[1]] += diff
            pt_results[team_names[0]] -= diff
        
        # retrieves index for each team to update the pool variables
        index0 = self.teams.index(team_names[0])
        index1 = self.teams.index(team_names[1])
        
        # updates pool's self.points
        self.points[index0] += pt_results[team_names[0]]
        self.points[index1] += pt_results[team_names[1]]
        
        # updates games_won
        self.games_won[index0] += game_results[team_names[0]]
        self.games_won[index1] += game_results[team_names[1]]
        
        # updates HTH
        # key = [team name, team name] (sorted)
        # value = point differential
        # if positive, team1 wins HTH
        # if negative, team0 wins HTH
        HTH_key=[index0,index1]
        HTH_key.sort()
        
        self.HTH[tuple(HTH_key)] = pt_results[self.teams[HTH_key[1]]]
                
    # returns point differential when two teams went HTH
    ##positive number means team with higher index WON point diff
    def get_HTH(self, indexes=0): 
        if indexes == 0:
            return self.HTH
        else:
            indexes.sort()
            return self.HTH[tuple(indexes)]
    
    def resolve_HTH(self):
        # for resolving HTH when the games won are the same (will work for multiple HTH)
        # will only change order if it's wrong
        # anytime rankings are re-sorted, HTH needs to be resolved again
        # also resolves multiple 2-way ties
        
        # goes through each entry in the rankings
        for i in range(4):
            
            # if games won is the same as the neighbor below in sorted ranking
            if self.rankings_sorted[i][2] == self.rankings_sorted[i+1][2]:
                
                # first, record indices of self and neighbor in teams
                indexes = [self.rankings_sorted[i][0],self.rankings_sorted[i+1][0]]
                
                # also record indices of self and neighbor in rankings
                indexes_rankings = [i, i+1]
                
                # if HTH > 0 then the team with the larger index(team) wins HTH
                # need to make sure that the team with the larger index also has a larger rankings index
                                    
                if self.get_HTH(indexes) > 0:
                    
                    if indexes[0]-indexes[1] < 0 and indexes_rankings[0]-indexes_rankings[1] < 0:
                                       
                        #switching the order
                        copy = self.rankings_sorted[i][:]
                        self.rankings_sorted[i] = self.rankings_sorted[i+1]
                        self.rankings_sorted[i+1] = copy
    
    # if there's a two-way tie, then resolve by head-to-head
    def resolve_ties(self):
        gameswon = [i[2] for i in self.rankings]
        modes = stats.mode(gameswon)

        if modes[1][0] == 2:
            self.resolve_HTH()
                
        elif modes[1][0] == 3:
            print("There are 3-way ties for " + self.court)
        
        # how to resolve 3-way ties?
        # currently 3-way ties are resolved by points
            
    # outputs list of teams **not ranked
    # uses tuples, (index, team_name, games_won, points)
    # returns a list of tuples
    def __get_unsorted_rankings(self):
        team_stats = []
        for i in range(len(self.teams)):
            team_stats.append((i, self.teams[i], self.games_won[i], self.points[i]))
            
        self.rankings = team_stats
        return self.rankings
    
    # sorts rankings by games won first
    # if there's a tie on games won, then resolve_ties, which includes resolve_HTH 
    def get_sorted_rankings(self):
        self.rankings_sorted = self.__get_unsorted_rankings()
        self.rankings_sorted = sorted(self.rankings_sorted, key=itemgetter(2,3), reverse=True)
        self.resolve_ties()
        return self.rankings_sorted
    
    # pretty prints sorted rankings
    def pp_rankings(self):
        rankings = self.get_sorted_rankings()
        
        print('\nPool:' + self.court)
        print ('Team Name\tGames Won\tPoints Overall')
        
        for item in rankings:
            print(item[1], '\t\t', item[2], '\t\t', item[3]) 

In [39]:
def main():
    ##Step1: create team names to feed into pools
    ##Step2: Create pools from list of team names
    ##Step3: Add games
    ##Step4: Print rankings by pool
    
    ##Step1: Create tournament
    ##Step2: Decide Gold, Silver, Bronze brackets
    
    teams = ['teama', 'teamb','teamc', 'teamd', 'teame']
    poolM1 = pool('M1', teams)
    poolM1.add_games(['teama','teamb'], [[21,19],[15,21]])
    poolM1.add_games(['teama','teamc'], [[22,20],[21,19]])
    poolM1.add_games(['teama','teamd'], [[22,20],[21,19]])
    poolM1.add_games(['teama','teame'], [[20,22],[21,19]])
    
    poolM1.add_games(['teamb','teamc'], [[22,10],[21,19]])
    poolM1.add_games(['teamb','teamd'], [[22,10],[21,19]])
    poolM1.add_games(['teamb','teame'], [[15,22],[19,21]])
    
    poolM1.add_games(['teamc','teamd'], [[22,24],[23,20]])
    poolM1.add_games(['teamc','teame'], [[24,22],[23,21]])
    
    poolM1.add_games(['teamd','teame'], [[22,15],[23,21]])
        
    poolM1.pp_rankings()
            
main()

There are 3-way ties for M1

Pool:M1
Team Name	Games Won	Points Overall
teama 		 6 		 4
teamb 		 5 		 23
teame 		 3 		 -4
teamd 		 3 		 -10
teamc 		 3 		 -13
